# Limpieza de Datos - Telecom X

## Propósito
En este cuaderno, se realiza la limpieza del dataset de Telecom X para garantizar que los datos estén listos para el análisis exploratorio. Esto incluye verificar valores ausentes, duplicados, errores de formato e inconsistencias en las categorías, además de desanidar las columnas en formato JSON.

## Tarea
1. Cargar los datos desde la URL de la API.
2. Desanidar las columnas JSON (`customer`, `phone`, `internet`, `account`).
3. Verificar duplicados y valores ausentes.
4. Identificar y corregir inconsistencias en las categorías, especialmente en `Churn`.
5. Guardar el dataset limpio para su uso posterior.

In [1]:
# Importación de bibliotecas
import pandas as pd
import requests
import json

# Carga de datos desde la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/main/TelecomX_Data.json"
response = requests.get(url)
data = response.json()

# Conversión a DataFrame
df = pd.DataFrame(data)

# 1. Desanidar columnas JSON
# Extraer datos de la columna 'customer'
customer_df = pd.json_normalize(df['customer'])
customer_df.columns = [f"customer_{col}" for col in customer_df.columns]

# Extraer datos de la columna 'phone'
phone_df = pd.json_normalize(df['phone'])
phone_df.columns = [f"phone_{col}" for col in phone_df.columns]

# Extraer datos de la columna 'internet'
internet_df = pd.json_normalize(df['internet'])
internet_df.columns = [f"internet_{col}" for col in internet_df.columns]

# Extraer datos de la columna 'account'
account_df = pd.json_normalize(df['account'])
account_df.columns = [f"account_{col}" for col in account_df.columns]

# Combinar los datos desanidados con las columnas originales
df_clean = pd.concat([df[['customerID', 'Churn']], customer_df, phone_df, internet_df, account_df], axis=1)

# 2. Verificación de duplicados
print("Número de filas duplicadas:", df_clean.duplicated().sum())

# 3. Verificación de valores nulos
print("\nValores nulos por columna:")
print(df_clean.isnull().sum())

# 4. Análisis de la columna 'Churn' para inconsistencias
print("\nValores únicos en 'Churn':")
print(df_clean['Churn'].value_counts())

# 5. Corrección de inconsistencias en 'Churn' (si aplica)
# Asumimos que 'Churn' debe ser binario ('Yes', 'No'). Investigamos el tercer valor.
# Si el tercer valor es un error (por ejemplo, espacio en blanco o valor inválido), lo corregimos.
df_clean['Churn'] = df_clean['Churn'].str.strip()  # Eliminar espacios en blanco
# Si el tercer valor no es 'Yes' o 'No', lo reemplazamos por el valor más común o lo marcamos como 'No' (decisión a validar)
df_clean['Churn'] = df_clean['Churn'].replace('', 'No')  # Ejemplo, ajustar según el valor encontrado

# 6. Conversión de tipos de datos (si es necesario)
# Por ejemplo, convertir 'account_Charges.Monthly' a float si no lo es
if 'account_Charges.Monthly' in df_clean.columns:
    df_clean['account_Charges.Monthly'] = pd.to_numeric(df_clean['account_Charges.Monthly'], errors='coerce')
if 'account_Charges.Total' in df_clean.columns:
    df_clean['account_Charges.Total'] = pd.to_numeric(df_clean['account_Charges.Total'], errors='coerce')

# 7. Inspección del DataFrame limpio
print("\nColumnas del DataFrame limpio:")
print(df_clean.dtypes)
print("\nPrimeras 5 filas del DataFrame limpio:")
print(df_clean.head())

# 8. Guardar el dataset limpio
df_clean.to_csv('TelecomX_Data_clean.csv', index=False)
print("\nDataset limpio guardado como 'TelecomX_Data_clean.csv'.")

Número de filas duplicadas: 0

Valores nulos por columna:
customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges.Monthly      0
account_Charges.Total        0
dtype: int64

Valores únicos en 'Churn':
Churn
No     5174
Yes    1869
        224
Name: count, dtype: int64

Columnas del DataFrame limpio:
customerID                    object
Churn                         object
customer_gender               object
customer_SeniorCitizen         int64
cu

## Descripción de la tarea
En esta sección, se desanidaron las columnas JSON (`customer`, `phone`, `internet`, `account`) para obtener variables individuales. Se verificaron duplicados (ninguno encontrado), valores nulos (ninguno identificado previamente) e inconsistencias en la columna `Churn`, que presentaba 3 valores únicos. Se corrigieron posibles errores en `Churn` y se aseguraron los tipos de datos correctos para variables numéricas como cargos mensuales y totales. El dataset limpio se guardó como `TelecomX_Data_clean.csv` para su uso en análisis posteriores.